## Workbook on solving ODEs with sympy

Recall our ODE from the [skydiver problem](https://github.com/fherwig/physmath248_pilot/tree/master/examples/Skydiver.ipynb). 

$$ v' = kv^2 -g $$

In [ ]:
## Analytic Solution in implicit form, using SymPy.
import sympy as sp

f=sp.Function('v')
x=sp.Symbol('x', real=True)
            
g,k=sp.symbols('g k')
ODE = sp.Eq( sp.Derivative(f(x), x), k*f(x)**2 - g )

print("We wish to get sympy to solve the differential equation: ")
sp.pprint(ODE)


In [ ]:
print("It is perfectly acceptable to ask sympy to solve an ODE given in the traditional form above.")
sp.pprint(sp.dsolve(ODE))

Sympy is happy with the ode in the above form.  It is also happy with it in the more traditional form
$$v' + g - kv^2 = 0$$
moreover, this allows us to omit the equality when calling *dsolve*.

In [ ]:
ODE = sp.Derivative(f(x), x) - k*f(x)**2 + g
sp.pprint(ODE)
sp.pprint(sp.dsolve(ODE))

In [ ]:
## Let's plot some solution curves. 

SOL = sp.dsolve(ODE)

SOLt = SOL.subs([(g, 9), (k,1)])
print("With g=9 and k=1 our solution curves are: \n")
sp.pprint(SOLt)

## We have too many variables for v to depend directly on x.
## Let's replace g and k with something reasonable. 

Notice sympy has inadvertently found the "wrong" solution, or something that looks like the wrong solution.  Since it did not take the absolute value when computing the anti-derivatives of 
$$\frac{1}{v-3}$$
i.e. it is using $\log(v-3)$ rather than $\log|v-3|$ this would appear to be the **wrong** answer.  

In a technical sense sympy is not making this exact mistake as sympy is considering the function $\frac{1}{z}$ to be a function on the complex plane, so its anti-derivative is a certain well-defined *branch of the logarithm* function.  Unfortunately, branches require choices and sympy is making the **wrong** choice of branch in the above expression! 

In [ ]:
print("log of -1")
sp.pprint(sp.log(-1))

print("\nlog of -i")
sp.pprint(sp.log(-sp.I))

This tells us sympy is using what is called as the *principal branch* of the logarithm function.

One way to fix this is to is to manipulate the symbolic expression directly, for example replacing the $v-3$ expression with $3-v$. We can do this with the *xreplace* call. 

Another (more systematic, requiring a knowledge of complex analysis) way to fix this problem would be to replace all occurances of $\log$ by the real part, i.e. $\log \longmapsto Re(\log)$. 

In [ ]:
## this expression still depends on a constant.  We need to extract the functional expression in (v,x) 
## to form a plot. So let's extract the left-hand side of the equation. 
sp.pprint(SOLt)
print("\nPerforming the replacement:")
repl_dict = {f(x)-3: 3-f(x)}
sp.pprint(SOLt.xreplace(repl_dict))
print("Ta da!")
SOLt = SOLt.xreplace(repl_dict)

In [ ]:
print("Recall how sympy thinks about symbolic expressions: ")
print(sp.srepr(SOLt))
print("\n")
sp.pprint(SOLt.args[0])
print("\nReplacing v(x) with y")

y = sp.Symbol('y')
SOLp = SOLt.args[0].subs(f(x), y)
sp.pprint(SOLp)


In [ ]:
from sympy.plotting import plot_implicit
from matplotlib import *
import matplotlib.pyplot as plt
import matplotlib.patches as patches
%matplotlib inline 

res = 2.9999999

curve1 = plot_implicit(sp.Eq(SOLp, 0), (x,-5,5), (y,-res,res), 
              adaptive=False, points=1800, 
              xlabel="time-axis", ylabel="v-axis", show=False)

curve2 = plot_implicit(sp.Eq(SOLp, 2), (x,-5,5), (y,-res,res), 
              adaptive=False, points=1800, 
              show=False)

curve3 = plot_implicit(sp.Eq(SOLp, -2), (x,-5,5), (y,-res,res), 
              adaptive=False, points=1800, 
              show=False)


curve1.extend(curve2)
curve1.extend(curve3)
curve1.show()

In [ ]:
## Or we could realize we could solve for x or y, rather than using the implicit plot. 
sp.pprint(SOLp)

print("Solve for x: \n")
sp.pprint(sp.solve(sp.Eq(SOLp, sp.Symbol('c')), x))
print("\nSolve for y: \n")
sp.pprint(sp.solve(sp.Eq(SOLp, sp.Symbol('c')), y))

Gy = sp.solve(sp.Eq(SOLp, sp.Symbol('c')), x)[0]
Fx = sp.solve(sp.Eq(SOLp, sp.Symbol('c')), y)[0]

In [ ]:
## this allows us to plot y as a function of x. 
sp.pprint(Gy)

G1 = Gy.subs(sp.Symbol('c'), 1)
print("After substitution: \n")
sp.pprint(G1)


In [ ]:
## plot x as a function of y
import numpy as np
res=2.9999
Ly = np.linspace(-res, res, 200)
G1n = sp.lambdify(y, G1, 'numpy')
plt.plot( G1n(Ly), Ly, 'b-')

In [ ]:
## plot y as a function of x
res = 3
F1 = Fx.subs(sp.Symbol('c'), 1)
sp.pprint(F1)
F1n = sp.lambdify(x, F1, 'numpy')

Lx = np.linspace(-5, 5, 200)
plt.plot( Lx, F1n(Lx), 'b-')

## More Generally -- Solving ODES with sympy

At present, sympy's ode solving algorithm is basically a big *cookbook*-style database of formulas.  The key computational task sympy must perform is to recognise what type of differential equation you have provided it.  

Once it recognises the form from its cookbook, it follows standard procedures, which usually amount to either computing anti-derivatives, Fourier transforms, power series, etc. 

**Sympy has algorithms to solve:**

* First order ODEs that are: 
     - separable differential equations
     - differential equations whose coefficients homogeneous of the same order.
     - exact differential equations.
     - linear differential equations
     - Bernoulli differential equations.

* Second order ODEs that are:
    - Liouville differential equations.

* n-th order ODEs that are:
    - linear homogeneous differential equation with constant coefficients.
    - linear inhomogeneous differential equation with constant coefficients using the method of undetermined coefficients.
    - linear inhomogeneous differential equation with constant coefficients using the method of variation of parameters.

Sympy also has algorithms to solve some [PDEs](http://docs.sympy.org/latest/modules/solvers/pde.html), Delay Differential Equations [DDEs](http://users.ox.ac.uk/~clme1073/python/PyDDE/) and pretty much any other kind of differential equation you can imagine. 

A key issue to solving differential equations is *determining what kind of differential equation* one is trying to solve. Once you *know* a differential equation is in (or can be put in) **form $X$**, and if sympy has an algorithm to solve differential equations of **form $X$**, then sympy can quickly give the answer.  

**example: **

*Exact differential equations* are of the form:
$$ f(x,y) \frac{dy}{dx} + g(x,y) = 0$$
with 
$$ \frac{\partial f}{\partial x} = \frac{\partial g}{\partial y} $$

Sympy has a routine that uses heuristics, based on its algorithms to solve algebraic equations, that check to see if your differential equation is of a type that it knows how to solve.  These tests can be sophisticated or not, and depends on the ease-of-applicability of the method.  For example, things like linear or separable ODEs are relatively easy to recognise, but exact differential equations can sometimes be subtle, due to the flexibility of the *integrating factor* technique. 

$$y' = - \frac{3xy+y^2}{x^2+xy}$$

is an exact ODE... or at least, it can be made to be exact with an integrating factor. 



In [ ]:
y=sp.Function('y')

sp.classify_ode( sp.Eq( y(x).diff(x) + ((3*x*y(x)+y(x)**2)/(x**2+x*y(x))), 0 ), y(x) )

The *classify_ode* routine returns the list of "types" of ODEs that the differential equation satisfies, which sympy knows how to solve.  If you find the default solution that sympy gives you is not satisfactory (perhaps the algebra is too complicated) you can ask sympy to solve your differential equation using another method, using the *hint* argument.

In [ ]:
sp.pprint(sp.dsolve(sp.Eq( y(x).diff(x) + ((3*x*y(x)+y(x)**2)/(x**2+x*y(x))), 0 ), 
                    hint="1st_homogeneous_coeff_subs_indep_div_dep_Integral"))

In [ ]:
sp.pprint(sp.dsolve(sp.Eq( y(x).diff(x) + ((3*x*y(x)+y(x)**2)/(x**2+x*y(x))), 0 ), 
                    hint="1st_homogeneous_coeff_subs_dep_div_indep_Integral"))

In [ ]:
sp.pprint(sp.dsolve(sp.Eq( y(x).diff(x) + ((3*x*y(x)+y(x)**2)/(x**2+x*y(x))), 0 ), 
                    hint="1st_homogeneous_coeff_subs_indep_div_dep"))

Generally sympy puts the technique considered to give the *most pleasant to work with* solutions at the top of the *classify_ode* list. 

In [ ]:
sp.pprint(sp.dsolve( ODE, hint="1st_power_series"))

In [ ]:
sp.pprint(sp.dsolve( ODE, hint="separable_Integral"))

In [ ]:
sp.pprint(sp.dsolve( ODE, hint="separable_Integral").doit() )

## More on exact ODEs

While Sympy did not recognise our ODE as exact (since it is not *literally* exact), we can still use sympy to look for an integrating factor -- something to turn it into an exact ODE... giving us another method to find solutions. 

Recall, *exact* ODEs are the ones that can be thought of as the gradient of a smooth function defined on the plane.  Any first-order $1$-dimensional ODE can be thought of as the flow lines of a vector field in the plane.  So we re-scale this vector field and try to show it is a gradient.

Our differential equation is:

In [ ]:
import sympy as sp
y=sp.Function('y')
x=sp.Symbol('x')

sp.pprint(sp.Eq( y(x).diff(x) + ((3*x*y(x)+y(x)**2)/(x**2+x*y(x))), 0 ) )

Differential equations of the form 

$$\frac{dy}{dx} + \frac{f(x,y)}{g(x,y)} = 0$$

have solutions that are orthogonal to the vector field

$$\vec F(x,y) = (f(x,y), g(x,y))$$

perhaps this is most easily seen by rewriting the differential equation as

$$ g(x,y) dy + f(x,y)dx = 0.$$

So if $\vec F$ was a gradient vector field, our solutions would be parametrizations of the level-sets of the potential function.  

In our case $\frac{\partial f}{\partial y} - \frac{\partial g}{\partial x} = x+y$ so $\vec F$ is *not* a gradient.  But we can modify the length of $\vec F$ and potentialls that new vector field would be a gradient.  This is the method of *integrating factors*. 

Let $u(x,y)$ be a positive function in the plane, and consider the problem of determining whether or not 
$$ u \vec F$$ 

is a gradient.  When the domain of $\vec F$ is *simply connected* (like a disc or a rectangle) this is equivalent to checking that the mixed partials agree. 

So we need to check that

$$\frac{\partial u}{\partial x} g - \frac{\partial u}{\partial y} f = u \left( \frac{\partial f}{\partial y} - \frac{\partial g}{\partial x} \right)$$

In [ ]:
## We implement the above. 

u = sp.Function('u')
y = sp.Symbol('y')
newEQ = sp.Eq( sp.diff(u(x,y)*(3*x*y+y**2), y) - sp.diff(u(x,y)*(x**2+x*y), x))
sp.pprint(newEQ)

In [ ]:
## Some standard ideas is to make u a function of only x or y.  Let's see what happens.
print("\nu(x) only.\n")
eqx = sp.Eq( sp.diff(u(x)*(3*x*y+y**2), y) - sp.diff(u(x)*(x**2+x*y), x))
sp.pprint(eqx)
sp.pprint(sp.solve(eqx, sp.diff(u(x), x) ))
eqx = sp.Eq( sp.diff(u(x), x), sp.solve(eqx, sp.diff(u(x), x) )[0] )
sp.pprint(eqx)

In [ ]:
print("\nu(y) only.\n")
eqy = sp.Eq( sp.diff(u(y)*(3*x*y+y**2), y) - sp.diff(u(y)*(x**2+x*y), x))
eqy = sp.Eq( sp.diff(u(y), y), sp.solve(eqy, sp.diff(u(y), y) )[0] )
sp.pprint(eqy)

In [ ]:
## let's see if sympy can solve either!

print("ODE types for eqx:")
sp.classify_ode( eqx, u(x) )

In [ ]:
print("\nODE types for eqy:")
sp.classify_ode( eqy, u(y) )

In [ ]:
## okay, so eqy it is!
sol = sp.dsolve(eqx, u(x))
sp.pprint(sol)

In [ ]:
## we don't need the c_1 term
sol = sol.subs( sp.Symbol('C1'), 1)
sp.pprint(sol)

In [ ]:
y=sp.Function('y')
## let's treat y as a function variable again...
sol = sol.subs(sp.Symbol('y'), y(x) )
#sp.pprint(sol)
u = sol.args[1]
#sp.pprint(u)
newODE = sp.Eq( u*(x**2+x*y(x))*sp.diff(y(x), x) + u*(3*x*y(x)+y(x)**2), 0 )
print("The new ODE is:\n")
sp.pprint(newODE)

sp.classify_ode( newODE, y(x) )


In [ ]:
sp.pprint( sp.dsolve( newODE, y(x) ) )